In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
import os, tempfile, shutil
#from IPython.display import display
from rdkit.Chem import Draw
import py3Dmol


mol=Chem.AddHs(Chem.MolFromSmiles('CCCOC1=CC=C(Cl)C(C)=C1'))
for i, a in enumerate(mol.GetAtoms()):
        a.SetAtomMapNum(i)
AllChem.EmbedMolecule(mol,randomSeed=10)
#mol
#Draw.MolToImage(mol, includeAtomNumbers=True,highlightAtoms=(1,2,3,4))
Draw.MolToFile(mol,"mol.png")




In [8]:
# code that should work in theory

from rdkit.Chem import rdMolTransforms
import altair as alt
import copy
import pandas as pd
from rdkit.Chem import rdForceFieldHelpers
from rdkit.Chem import ChemicalForceFields

conformer=mol.GetConformer(0)

m2=copy.deepcopy(mol)
mp2 = AllChem.MMFFGetMoleculeProperties(m2)
energy=[]
confid=0
angles=range(0,370,10)
for angle in angles:
    confid+=1
    ff2 = AllChem.MMFFGetMoleculeForceField(m2, mp2)
    ff2.MMFFAddTorsionConstraint(1,2,3,4, False, angle - .1, angle + .1, 10000.0)
    ff2.Minimize()
    energy.append(ff2.CalcEnergy())

    xyz=ff2.Positions()
    new_conf = Chem.Conformer(mol.GetNumAtoms())
    for i in range(mol.GetNumAtoms()):
        new_conf.SetAtomPosition(i, (m2.GetConformer(-1).GetAtomPosition(i)))
    new_conf.SetId(confid)
    mol.AddConformer(new_conf)


dfrdkit = pd.DataFrame({'angle':angles, 'energy':energy})
alt.Chart(dfrdkit).mark_line(point=True,interpolate="natural").encode(
    alt.X('angle:Q',
        scale=alt.Scale(zero=False)
    ),
    alt.Y('energy:Q',
        scale=alt.Scale(zero=False)
    )
).interactive()

alt.Chart(...)

In [126]:
from rdkit.Chem import rdMolTransforms
import altair as alt
import copy
import pandas as pd
from rdkit.Chem import rdForceFieldHelpers
from rdkit.Chem import ChemicalForceFields

conformer=mol.GetConformer(0)
mp = AllChem.MMFFGetMoleculeProperties(mol)
mp.SetMMFFAngleTerm(True)
mp.SetMMFFEleTerm(True)
mp.SetMMFFOopTerm(False)
mp.SetMMFFVdWTerm(True)
mp.SetMMFFStretchBendTerm(True)
ffm = AllChem.MMFFGetMoleculeForceField(mol, mp)
m2=copy.deepcopy(mol)
energy=[]
confid=0
for angle in range(0,370,10):
    confid+=1
    ff2 = AllChem.MMFFGetMoleculeForceField(m2, mp)
    ff2.MMFFAddTorsionConstraint(1,2,3,4, False, angle - .1, angle + .1, 10000.0)
    ff2.Minimize()
    
    xyz=ff2.Positions()
    new_conf = Chem.Conformer(mol.GetNumAtoms())
    for i in range(mol.GetNumAtoms()):
        new_conf.SetAtomPosition(i, (m2.GetConformer(-1).GetAtomPosition(i)))
    new_conf.SetId(confid)
    mol.AddConformer(new_conf)
    energy.append(ffm.CalcEnergy(ff2.Positions()))
dfrdkit = pd.DataFrame({'angle':range(0,370,10), 'energy':energy})
alt.Chart(dfrdkit).mark_line(point=True,interpolate="natural").encode(
    alt.X('angle:Q',
        scale=alt.Scale(zero=False)
    ),
    alt.Y('energy:Q',
        scale=alt.Scale(zero=False)
    )
).interactive()

alt.Chart(...)

In [3]:
from ipywidgets import interact, interactive, fixed
patt = Chem.MolFromSmarts('c1ccccc1');patt
match = mol.GetSubstructMatch(patt)

AllChem.AlignMolConformers(mol,atomIds=match)


def drawit(m,p,confId):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p.removeAllModels()
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    #p.zoomTo()
    return p.show()
viewer = py3Dmol.view(width=500, height=500)
mb = Chem.MolToMolBlock(mol,confId=0)
viewer.addModel(mb,'sdf')
viewer.setStyle({'stick':{}})
viewer.zoomTo()
#viewer.show()
conformerIds=[conf.GetId() for conf in mol.GetConformers()]
interact(drawit, m=fixed(mol),p=fixed(viewer),confId=(0,mol.GetNumConformers()-1))




interactive(children=(IntSlider(value=0, description='confId', max=0), Output()), _dom_classes=('widget-intera…

<function __main__.drawit(m, p, confId)>

In [131]:
#write the xtb input file:
energy=[]
import os
for idx,deg in enumerate(range(0,370,10)):
  print(deg)
  w = Chem.SDWriter('mol.sdf')
  w.write(mol,confId=idx+1)
  w.close()

  atoms = '2,3,4,5'
  fh = open("dihedral_scan.inp","w")
  fh.write("""$constrain
    force constant=1.05
    dihedral: {},{}
  $end""".format(atoms,float(deg)))
  fh.close()
  os.system("export OMP_STACKSIZE=48G && export OMP_NUM_THREADS=12,1 && xtb mol.sdf --opt vtight --charge 0 --input dihedral_scan.inp")

  sdr=Chem.SDMolSupplier("xtbopt.sdf")
  for xtbmol in sdr:
      energy.append(xtbmol.GetProp("total energy / Eh"))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360


In [132]:
import numpy as np
angle=np.array(range(0,370,10))

dfxtb = pd.DataFrame({'angle':angle, 'xtb':energy,'MMFF':dfrdkit["energy"]})
#data = dfxtb.melt('angle')

base = alt.Chart(dfxtb).encode(
    alt.X('angle:Q', axis=alt.Axis(title=None),scale=alt.Scale(domain=[0,360,350]))
)
line1 = base.mark_line(stroke='#5276A7', interpolate='natural').encode(
    alt.Y('xtb:Q',
          axis=alt.Axis(title='xtb energy', titleColor='#5276A7'),scale=alt.Scale(zero=False))
)
line2 = base.mark_line(stroke='#57A44C', interpolate='natural').encode(
    alt.Y('MMFF:Q',
          axis=alt.Axis(title='MMFF energy', titleColor='#57A44C'),scale=alt.Scale(zero=False))
)
alt.layer(line1, line2).resolve_scale(
    y = 'independent'
)



alt.LayerChart(...)

ModuleNotFoundError: No module named 'matplotlib'